In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# List files to ensure correct path
:contentReference[oaicite:2]{index=2}

# Load data
:contentReference[oaicite:3]{index=3}
:contentReference[oaicite:4]{index=4}
:contentReference[oaicite:5]{index=5}

:contentReference[oaicite:6]{index=6}


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

le = WordNetLemmatizer()
stops = set(stopwords.words('english'))

def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r'\d+','', txt)
    txt = re.sub(r'[^\w\s]','', txt)
    tokens = nltk.word_tokenize(txt)
    tokens = [le.lemmatize(t) for t in tokens if t not in stops]
    return " ".join(tokens)

for col in ['prompt', 'response_a', 'response_b']:
    train[col] = train[col].astype(str).apply(clean_text)
    test[col] = test[col].astype(str).apply(clean_text)


In [ ]:
MAX_FEAT = 300

tfidf_prompt = TfidfVectorizer(max_features=MAX_FEAT)
X_prompt = tfidf_prompt.fit_transform(train['prompt'])
X_prompt_test = tfidf_prompt.transform(test['prompt'])

tfidf_resp = TfidfVectorizer(max_features=MAX_FEAT)
X_a = tfidf_resp.fit_transform(train['response_a'])
X_b = tfidf_resp.transform(train['response_b'])
X_a_test = tfidf_resp.transform(test['response_a'])
X_b_test = tfidf_resp.transform(test['response_b'])

from scipy.sparse import hstack
X_train = hstack([X_prompt, X_a, X_b])
X_test = hstack([X_prompt_test, X_a_test, X_b_test])

y_train = train['winner_model'].map({'a':0,'b':1,'tie':2}).values


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=500, multi_class='multinomial', solver='saga')
model.fit(X_tr, y_tr)

y_pred_proba = model.predict_proba(X_val)
print("Validation Log Loss:", log_loss(y_val, y_pred_proba))


In [ ]:
test_pred_proba = model.predict_proba(X_test)
sub = pd.DataFrame({
    'id': test['id'],
    'winner_model_a': test_pred_proba[:, 0],
    'winner_model_b': test_pred_proba[:, 1],
    'winner_tie': test_pred_proba[:, 2]
})
sub.to_csv('submission.csv', index=False)
print("submission.csv saved.")
